# Quantized Neural Network
## 07 Efficient Integer Inference
by [Soon Yau Cheong](http://www.linkedin.com/in/soonyau)

One advantage of Google's quantization scheme is that it can run efficiently on existing processors including a fixed-point (inter) only processor. Compare with floating point processor, the former is usually smaller in size and more power efficient. However, from computation complexity, it actually require a lot more arithemtic operations than running in full floating point mode due to the need to de-quantize the weights and activation, perform the computation, then requantize it back into low precision integers. Although in general the inference speed should improve due to reduce memory traffic but this additional computation is a big set back. In this tutorial, we'll look at tricks to reduce the computation complexity.

### Requantization
Now let's revisit the quantization equation
\begin{equation}
x_q = \frac{x}{\Delta_x} + z_x
\end{equation}
and dequantization equation

\begin{equation}
\hat{x} = (x_q - z_x )\Delta_x
\end{equation}

(Simplified) convolutional equations:
\begin{equation}
y = \sum_N^i w_i x_i +b
\end{equation}

Now dequantize $x_q$, $w_q$ and $b_q$ to perform convolution in integers
\begin{equation}
y_q = \sum_N^i (w_{q,i} - z_w )\Delta_w (x_{q,i} - z_x )\Delta_x + (b_q - z_b )\Delta_b
\end{equation}

Let's rearrange the equation and denote the signed integer $w'_{i} = w_{q,i} - z_w$ 
\begin{equation}
y_q = \Delta_w \Delta_x \sum_N^i w'_i x'_i + \Delta_b b'
\end{equation}

Last step is to quantize the result back into integer
\begin{equation}
\hat{y} = \frac{1}{\Delta_y}(\Delta_w \Delta_x \sum_N^i w'_i x'_i + \Delta_b b') +  z_y \\
\hat{y} = \frac{\Delta_w \Delta_x}{\Delta_y} \sum_N^i w'_i x'_i + \frac{\Delta_b}{\Delta_y} b' +  z_y \\
\end{equation}

That looks a bit complex, now we'll begin simplification. As opposed to weights that are quantized to uint8, biases are quantized to int32 and we don't really worry about losing resolution due to scaling. Therefore, $\Delta_b$ is chosen to be equal to $\Delta_w \Delta_x$, and the above equation becomes
\begin{equation}
\hat{y} = (M \sum_N^i w'_i x'_i) +  (Mb' +  z_y )
\end{equation}
where
\begin{equation}
M = \frac{\Delta_w \Delta_x}{\Delta_y}
\end{equation}
which can be simplified again to 

\begin{equation}
\hat{y} = (M \sum_N^i w'_i x'_i) +  B
\end{equation}
where
\begin{equation}
B = (Mb' +  z_y )
\end{equation}

Instead of performing de-quantization for every activation, weights, and biases, and quantize the results, we now only need to perform 2 additional operations compared with the original convolutional equations (note that M and B can be pre-calculated from constants)
- minus offset from quantized integer to get signed integer
- multiply with dot product with M

### Removing Offset
One potential improvement to the scheme is to avoid the computation of signed integer, the way to do it would be to get rid of the offsets altogether which is exactly the case of biases where the offset is always 0 and quantized to int32. If we were to do that to weights, we will need to add another signed bit which grow the bidwith, for example, instead of using uint8, we now use int9 and remove the offset. Alternatively, we reduce 1-bit in our quantization scheme to int8, as we have shown previously with Cifar10, there is no accuracy loss going from 8-bit to 7-bit. 

### Multiplier
All variables in the equations are now integer-only, apart from M which can be floating point and how are we going to deal with that. One popular trick to port floating point to fixed point in embedded system is to decompose the floating point to a pair of fixed integer and shift operations. In [Google's paper](https://arxiv.org/pdf/1712.05877.pdf), they use the term
\begin{equation}
M = 2^{-n}M_o
\end{equation}
where $M_o$ is integer and n is the number of right shift operation. For those who are not familiar with bitshift operation, shifting integer by 1 bit to the right is equivalent to dividing by 2, 2-bits is divided by 4 and so on. Now let's jump into some example to demonstrate. I won't write the full routine to compute convolution but will use actual quantization parameters of Mobilenet to demonstrate the calculation.

In [4]:
import os
import sys
import numpy as np
import tensorflow as tf

import utils
print("Tensorflow", tf.__version__)
print("Python", sys.version)

model_path = 'models/mobilenet_v1/mobilenet_v1_1.0_224_quant.tflite'

interpreter = tf.contrib.lite.Interpreter(model_path=model_path)
                                         
interpreter.allocate_tensors()
num_layer = 89
for i in range(num_layer):
    detail = interpreter._get_tensor_details(i)
    # Let's pick the first convolutional layer where input is RGB image
    if "Conv2d_0" in detail['name']:
        print(i, detail['name'])

Tensorflow 1.10.0
Python 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
6 MobilenetV1/MobilenetV1/Conv2d_0/Conv2D_Fold_bias
7 MobilenetV1/MobilenetV1/Conv2d_0/Relu6
8 MobilenetV1/MobilenetV1/Conv2d_0/weights_quant/FakeQuantWithMinMaxVars


In [16]:
scale_b, offset_b = interpreter._get_tensor_details(6)['quantization']
print("bias", scale_b, offset_b)
scale_y, offset_y = interpreter._get_tensor_details(7)['quantization']
print("y", scale_y, offset_y)
scale_w, offset_w = interpreter._get_tensor_details(8)['quantization']
print("w", scale_w, offset_w)
scale_x, offset_x = 1./128, 127
print("x", scale_x, offset_x)

bias 0.00017052092880476266 0
y 0.023528477177023888 0
w 0.02182667888700962 151
x 0.0078125 127


Now let's check if $\Delta_b == \Delta_w \Delta_x$.

In [18]:
print(scale_w*scale_x)
print(scale_b)

0.00017052092880476266
0.00017052092880476266


Our objective now is to find n and corresponding $Mo$ using equation
\begin{equation}
Mo = round(2^n \times M)
\end{equation}
The multiplication between M and say P will now becomes
\begin{equation}
M\times P \approx (M_o \times P) >> n
\end{equation}

In [59]:
# We now take the weights of one of the filter
w = interpreter.get_tensor(8)
w_int = w[2,:,:,:].astype(np.int) - int(offset_w)
print(w_int)

# Generate random input
x_int = np.random.randint(0,255,w_int.shape) - int(offset_x)

# Perform dot product
P = np.sum(np.multiply(x_int, w_int))

[[[ 14 -11 -13]
  [ 24 -14 -16]
  [ 10  -2  -6]]

 [[ 28 -13 -14]
  [ 39 -13 -15]
  [  9  -8  -8]]

 [[ 13  -6  -9]
  [ 14  -9  -8]
  [ -3   0  -4]]]


In [74]:
# Now calculate M
M = scale_w*scale_x/scale_y
print("M=%.16f P=%d, MxP=%f"%(M, P, M*P))

def multiply(n, M, P):
    # floting point operation
    result = M*P
    
    # calculate Mo
    Mo = int(round(2**n*M))
    
    # integer arithmetic only
    approx_result = (Mo*P)>>n
    
    print("n=%d, Mo=%d, approx=%f, error=%f"%\
          (n, Mo, approx_result, result-approx_result))
    return approx_result

# Our objective now is to find n and Mo
for n in range(1, 16):
    multiply(n, M, P)

M=0.0072474273418460 P=7091, MxP=51.391507
n=1, Mo=0, approx=0.000000, error=51.391507
n=2, Mo=0, approx=0.000000, error=51.391507
n=3, Mo=0, approx=0.000000, error=51.391507
n=4, Mo=0, approx=0.000000, error=51.391507
n=5, Mo=0, approx=0.000000, error=51.391507
n=6, Mo=0, approx=0.000000, error=51.391507
n=7, Mo=1, approx=55.000000, error=-3.608493
n=8, Mo=2, approx=55.000000, error=-3.608493
n=9, Mo=4, approx=55.000000, error=-3.608493
n=10, Mo=7, approx=48.000000, error=3.391507
n=11, Mo=15, approx=51.000000, error=0.391507
n=12, Mo=30, approx=51.000000, error=0.391507
n=13, Mo=59, approx=51.000000, error=0.391507
n=14, Mo=119, approx=51.000000, error=0.391507
n=15, Mo=237, approx=51.000000, error=0.391507


There you go. We have shown that by decomposing M to n and Mo, we can approximate floating point multiplication using only integer multiplication and bit shifting. We notice that higher n gives better approximation with lesser error. However, the choice of n is constrained by the bitwidth of hardware processor and the variables. 

In [81]:
#For a 3x3x3 conv kernel, the dot product must accomodate maximum value of
max_val = 3*3*3*255*255

# which require bit numbers of
numbits = round(np.log2(max_val)+1) # +1 for signed bit
print(numbits)

22.0


If a processor internal registers could only hold 32-bit value at a time, then we will only have maximum 32-22 = 10 bits left for Mo, from there we can select our n. We'll need to perform system-wise analysis to find out the optimal values that do not cause arithemtic operations to overflow. In CNN, the worst case scenario would be the convolutional kernel with largest dimension, in Mobilenet, that would the last conv layer with 1x1x1024 weights which I'll leave as exercise to calculate the number of bits required. Anyway, in the most unlikely even of having both activations and weights to be all 255 which is the worst case scenario we assume, something has gone terribly wrong. Therefore, we can relax our requirement by 1 bit or 2 in calculating for normal operation.

## What's Next?
You now have all the skills to train a quantized network, convert the graph and export the quantized weights for inference. For motivated readers, you can now implement the inference by writing own software/FPGA code which isn't very difficult given there really are only 2 type of layers in Mobilenet - depthwise and pointwise convolution. Next time, we'll look at how to run TFLite on Android.